# Using the OPERA DIST_ANN to Track Wildfire Impact
**This notebook ----------.**

**<font color='red'>Note: This notebook uses provisional products, which may differ slightly from operational products. Please refer to [DIST product specification](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_DIST_HLS.pdf) for more information. </font>**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Library Imports
import math
import xarray as xr
import hvplot.xarray
import geoviews as gv
import pyproj
import numpy as np
import geopandas as gpd
import holoviews as hv
import panel.widgets as pnw
import datetime
from datetime import date, timedelta, datetime

import folium
from folium import plugins

import leafmap.leafmap as leafmap

import rioxarray
from skimage import io 
import matplotlib.pyplot as plt

from bokeh.models import FixedTicker
hv.extension('bokeh')

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')
from src.dist_utils import *

### Background

A series of fires affected counties of northern California during the 2021 and 2022 wildfire seasons. We will be looking at a DIST_ANN tile that captures the cumulataive vegetation disturbance associated with a subset of these fires located in the region of Klamath National Forest in western Siskiyou County:

| Name | Dates |
| ---  | --- |
| Antelope | 8/1/2021 - 1/3/2022 |
| Tennant | 6/28/2021-1/3/2022 |
| Lava | 6/25/2021-10/26/2021 |
| Gulch | 3/12/2022-3/14/2022 |
| Mountain | 9/2/2022-10/2/2022 |
| Mckinney | 7/29/2022-9/7/2022 |
| Alex | 7/31/2022 - 8/3/2022 |
| Mill | 9/2/2022-9/11/2022|  
| Coyote | 9/7/2022-9/8/2022| 

*Data from the [California Data Portal](https://data.ca.gov/dataset/california-fire-perimeters-all)*

Specifically, we will be visualizing the spatial and temporal impacts of the fires on vegetation changes caused by these fires using the **OPERA DIST_ANN product**, a pixel-wise summary of annual vegetation disturbance to demonstrate how the product may be used to quantify the spatial magnitude of wildfire impacts and vegetation regrowth. 

**Probably cut the below:**
Among these fires, the most spatially destructive was the McKinney and  McKinney Fire was a destructive wildfire that burned in the Klamath National Forest in western Siskiyou County, California, in the United States as part of the 2022 California wildfire season. (Update this its a copy/paste from wiki). The fire ignited at approximately 2:15PM (PST) on July 29, 2022 and destroyed ~60,000 acres in the next x days. The fire and was considered 100% contained as of October 1, 2022. 


*The DIST_ANN product used here is a provisional product with an observation date exceeding the operational 1-year period.*

In [3]:
# Load the DIST_ANN bands of interest
data_dir = 'https://glad.umd.edu/projects/opera/SEP/DIST-ANN/10/T/E/M/2022/OPERA_L3_DIST-ANN-HLS_T10TEM_2022_2023136T210508Z_30_v0_'
bandlist = ['VEG-DIST-STATUS', 'VEG-HIST', 'VEG-IND-MAX','VEG-ANOM-MAX', 'VEG-DIST-CONF', 'VEG-DIST-DATE', 
            'VEG-DIST-COUNT', 'VEG-DIST-DUR', 'VEG-LAST-DATE', 'GEN-DIST-STATUS', 'GEN-ANOM-MAX', 'GEN-DIST-CONF',
            'GEN-DIST-DATE', 'GEN-DIST-COUNT', 'GEN-DIST-DUR', 'GEN-LAST-DATE']
bandpath = f"{data_dir}%s.tif"

Let's first have a look at the overall area of affected forest. The Vegetation Disturbance Status (VEG-DIST-STATUS) raster tile denotes pixels which indicate vegetation disturbance (loss) over the calendar year (years?). 

## Harmonized Landsat and Sentinel (HLS)
The OPERA Disturbance product is derived from HLS. The time series of HLS is what provides the quantification of vegetation disturbance.

In [4]:
authenticators = check_netrc()
make_manager(authenticators)

netrc exists and includes NASA Earthdata login credentials.


In [5]:
# Stream HLS - This is somewhat involved probably around 5 cells. Need to read in RGB, or maybe NIR for false-co?

In [6]:
# Leafmap cant stream from Earthdata, it would seem. And it cant do cogs in memory. 

In [7]:
# Seems like have to stream and write to file then load? Seems like that really shouldn't be necessary...

In [8]:
### For the time being, I am going to simply plot the thumbnails side by side, which does not require authentication.

In [11]:
### Local files
leafmap.split_map(
    left_layer="../HLS/false_color_7_21_2022.tif",
    right_layer="../HLS/false_color_9_23_2022.tif",
    left_label="07/21/2022",
    right_label="09/23/2022",
    label_position="bottom",
    center=[42.0, -122.3],
    zoom=9,
)

Map(center=[42.0, -122.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

Zoom in on the area affected by the McKinney Fire

In [7]:
### Local files
leafmap.split_map(
    left_layer="../HLS/false_color_7_21_2022.tif",
    right_layer="../HLS/false_color_9_23_2022.tif",
    left_label="07/21/2022",
    right_label="09/23/2022",
    label_position="bottom",
    center=[41.80, -122.90],
    zoom=11,
)

Map(center=[41.8, -122.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### **<font color='red'> -- Do not modify any of the code below -- </font>**

In [10]:
# Global variables
total_pixel_count = 3660 * 3660
pixel_area = 30 * 30
ref_date = datetime(2020, 12, 31)
starting_day = (starting_date - ref_date).days
ending_day = (ending_date - ref_date).days + 1
bandpath = f"{data_dir}%s.tif"

### Temporal tracking

In [9]:
# Filter dates to focus only the relevant timeframe, e.g. McKinney Fire start date: Day 575 (July 29, 2022)
starting_date = datetime(2022,8,1) # 29Jul2022 McKinney started
ending_date = datetime(2022,8,15)  # 15Aug2022 Acquisition date of the input HLS
n = 2  # Interval day for the slider. e.g., every 2 days

data_dir ='https://opera-provisional-products.s3.us-west-2.amazonaws.com/DIST/DIST_HLS/WG/DIST-ALERT/McKinney_Wildfire/OPERA_L3_DIST-ALERT-HLS_T10TEM_20220815T185931Z_20220817T153514Z_S2A_30_v0.1/OPERA_L3_DIST-ALERT-HLS_T10TEM_20220815T185931Z_20220817T153514Z_S2A_30_v0.1_'
bandlist = ['VEG-DIST-STATUS', 'VEG-DIST-DATE', 'VEG-ANOM-MAX']

# Maximum Anomaly threshold 
anom_threshold = 15 # filter pixels with maximum anomaly < 15%

# Focus the wildfire area extent calculation on a small region within the DIST product tile
bounds = [2000,3000,0,1000]   # upper-left y-pixel, lower-left y-pixel, upper-left x-pixel, lower-right x-pixel

In [11]:
da_alert, crs = stack_bands(bandpath, bandlist)

# Create basemap
base = gv.tile_sources.EsriTerrain.opts(width=1000, height=1000, padding=0.1)

In [12]:
# Create a multidimensional array as a function of time and calculate the area of the burn as it evolves through time
area_coverage = time_and_area_cube(da_alert.z.sel({'band':1}), da_alert.z.sel({'band':2}), da_alert.z.sel({'band':3}),
                                   anom_threshold, pixel_area, bounds, starting_day, ending_day, ref_date, step=n)

In [13]:
## Visualize the evolution of the fire overlaid on a basemap

# Create a basemap
base = gv.tile_sources.EsriNatGeo()

# Plot wildfire area extent time series
area_coverage_slider = area_coverage.z.interactive.sel(time=pnw.DiscreteSlider).hvplot(x='lon', 
                                                                                       y='lat', 
                                                                                       crs=crs, 
                                                                                       kind='image', 
                                                                                       rasterize=True, 
                                                                                       dynamic=True,
                                                                                       cmap='hot_r', 
                                                                                       aspect='equal', 
                                                                                       frame_width=600, 
                                                                                       frame_height=600).opts(active_tools=['wheel_zoom'],
                                                                                                              xlabel='Longitude', 
                                                                                                              ylabel='Latitude',  
                                                                                                              clim=(0,4), 
                                                                                                              colorbar_opts={'ticker': FixedTicker(ticks=[0, 1, 2, 3, 4])}, 
                                                                                                              alpha=0.9)
area_coverage_slider * base

ModuleNotFoundError: No module named 'jupyter_bokeh'

## DIST-ANN

In [14]:
# Create geocube of stacked bands
da, crs = stack_bands(bandpath, bandlist)

# Create basemap
base = gv.tile_sources.EsriTerrain.opts(width=1000, height=1000, padding=0.1)

### Visualize the annual summary VEG_DIST_STATUS
The 'VEG_DIST_STATUS' layer indicates 

In [15]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_dist_status = da.z.where(da['z']!=255).sel({'band':1})

color_key = {
    "Confirmed, <50% ongoing": "#ffffb2",
    "Confirmed, <50% completed": "#f45629",
    "Confirmed, ≥50% ongoing": "#feb751",
    "Confirmed, ≥50% completed": "#bd0026",
}

levels = 4
ticks = [2.5, 3.5, 4.5, 5.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

veg_dist_status.where(veg_dist_status!=0).hvplot.image(x='longitude', 
                             y='latitude', 
                             crs=crs, 
                             rasterize=True,
                             dynamic=True, 
                             aspect='equal', 
                             frame_width=500, 
                             frame_height=500,
                             clim=(2,6), alpha=0.8).opts(title=f"VEG_DIST_STATUS", xlabel='Longitude', 
                                                         ylabel='Latitude', color_levels = levels, 
                                                         cmap=tuple(color_key.values()),
                                                         colorbar_opts={'ticker': ticker, 'major_label_overrides':labels}) * base

ModuleNotFoundError: No module named 'jupyter_bokeh'

:DynamicMap   []

## Thresholding
There is a lot of noise and these likely correspond to clouds. We can supplemen 'filter' this by the confidence layer. Do do this we query the confidence layer to retain only the pixels over a user specified threshold.

### Ideas

- Compute a new band that is the VEG_DIST_STATUS band but includes a confidence threshold from VEG_DIST_CONF (hopefully this would remove some of the noise)
- Use xarray or other geostats package to compute affected area for each group, including the area that has recovered.
- Get the duration of the disturbance for these pixels (VEG_DIST_DUR). Conditional raster math of some sort would be able to get a new raster containing only pixels that are recovered and how long that took to occur.
- Investigate spatial patterns in regrowth rates. Do we see more around the edges? Is there more in the N/S/E/W? 
- Add topography data to further flesh out why regrowth may be occuring preferentially in some areas (if at all).
- **Is it possible to scatterplot any of the variables to gain any additional insight?**

### Observations
- It seems like there has been very little 'full recovery' per the VEG_DIST_STATUS layer for any fire-affected pixels. (**Maybe this will require the 'generic' layers to get more at the amount of recovery.**)
- Looking at the Generic Disturbance Status (GEN-DIST-STATUS), many pixels are classified as 2 (confirmed, <50% ongoing) whereas the same pixels are classified as 4 (confirmed, >50% ongoing). So there is a difference in how the algorithms classify the pixels between the datasets...